<a href="https://colab.research.google.com/github/stevenbowler/LLama2HealthCareChatBot/blob/master/LLama2HealthCareChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!pip install  -qU \
langchain \
torch \
accelerate \
transformers \
sentence_transformers \
streamlit \
streamlit_chat \
faiss-cpu \
altair \
tiktoken \
huggingface-hub \
bitsandbytes \
ctransformers

In [72]:
!pip install pypdf

In [73]:
!pip install streamlit streamlit_chat langchain


In [74]:
import streamlit as st
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader as PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory


In [75]:
# add google drive for data files
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:

#load the pdf files from the path
loader = DirectoryLoader('/content/drive/MyDrive/ColabStuff',glob="2023-07-TMPPM.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

In [77]:

#split text into chunks
text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks = text_splitter.split_documents(documents)

In [78]:

#create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                   model_kwargs={'device':"cuda"})


In [79]:
!pip install faiss-gpu

In [80]:

#vectorstore
vector_store = FAISS.from_documents(text_chunks,embeddings)


In [81]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = ''
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [ ]:
filename = '/content/drive/MyDrive/ColabStuff/LLama2HealthCareChatBot.sav'
!touch filename
!pwd
# model.save_pretrained(filename)
# torch.save(model, filename)


# pickle.dump(model, open(filename, 'wb'))

# Make the function picklable
# def foo():
#   return model

# with open(filename, 'wb') as f:
#   unpickled_foo = pickle.load(f)


# pickle.dumps(model.forward)

/content


In [82]:
from huggingface_hub import notebook_login
notebook_login()  # then wait at least 1 minute before next step for auth

In [83]:
!pip install chatdocs

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Aug 14 08:55:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    52W / 400W |   5735MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [84]:
from langchain.memory import ConversationBufferMemory

In [85]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [90]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
print(transformers.__file__)

/usr/local/lib/python3.10/dist-packages/transformers/__init__.py


In [ ]:
from langchain.llms import CTransformers

In [92]:
from transformers import AutoModelForCausalLM

In [ ]:
!curl -L https://thebloke.github.io/Llama-2-7B-Chat-GGML/llama-2-7b-chat.ggmlv3.q4_0.bin -o llama-2-7b-chat.ggmlv3.q4_0.bin

https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9115  100  9115    0     0    97k      0 --:--:-- --:--:-- --:--:--   98k


In [96]:
!pip install auto_gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/1b/79/5a3a7d877a9b0a72f528e9977ec65cdb9fad800fa4f5110f87f2acaaf6fe/auto_gptq-0.3.2.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto_gptq from https://files.pythonhosted.org/packages/1b/79/5a3a7d877a9b0a72f528e9977ec65cdb9fad800fa4f5110f87f2acaaf6fe/auto_gptq-0.3.2.tar.gz has inconsistent version: expected '0.3.2', but metadata has '0.3.2+cu118'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/3f/5c/28d57f833498e014e9d066ea0199a503b5e59d4c8e8e701a460223e143da/auto_gptq-0.3.1.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto_gptq from https://files.pythonhosted.org/packages/3f/5c/28d57f833498e014e9d066ea0199a503b5e59

In [97]:
# from the bloke
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "gptq_model-4bit-128g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

In [98]:
# create llm

# llm = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")


llm = CTransformers(model=model,model_type="llama",
                    config={'max_new_tokens':128,'temperature':0.01})

# llm = CTransformers.from_pretrained(model)
# llm.load_model(model)
# llm.set_model_type("llama")
# llm.set_config({'max_new_tokens':128,'temperature':0.01})

KeyError: ignored

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [94]:
chain = ConversationalRetrievalChain.from_llm(llm=llm,chain_type='stuff',
                                              retriever=vector_store.as_retriever(search_kwargs={"k":2}),
                                              memory=memory)


ValidationError: ignored

In [ ]:

st.title("HealthCare ChatBot 🧑🏽‍⚕️")
def conversation_chat(query):
    result = chain({"question": query, "chat_history": st.session_state['history']})
    st.session_state['history'].append((query, result["answer"]))
    return result["answer"]

def initialize_session_state():
    if 'history' not in st.session_state:
        st.session_state['history'] = []

    if 'generated' not in st.session_state:
        st.session_state['generated'] = ["Hello! Ask me anything about 🤗"]

    if 'past' not in st.session_state:
        st.session_state['past'] = ["Hey! 👋"]

def display_chat_history():
    reply_container = st.container()
    container = st.container()

    with container:
        with st.form(key='my_form', clear_on_submit=True):
            user_input = st.text_input("Question:", placeholder="Ask about your Mental Health", key='input')
            submit_button = st.form_submit_button(label='Send')

        if submit_button and user_input:
            output = conversation_chat(user_input)

            st.session_state['past'].append(user_input)
            st.session_state['generated'].append(output)

    if st.session_state['generated']:
        with reply_container:
            for i in range(len(st.session_state['generated'])):
                message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="thumbs")
                message(st.session_state["generated"][i], key=str(i), avatar_style="fun-emoji")


NameError: ignored

In [ ]:

# Initialize session state
initialize_session_state()

In [ ]:
# Display chat history
display_chat_history()